In [ ]:
import sys
import os
from pathlib import Path
import pandas as pd
from sklearn.metrics import mean_absolute_error

ROOT = Path(os.getcwd()).resolve().parents[0] 
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

from pipelines.inference_pipeline.inference import predict

sample_path = ROOT / "data/raw/holdout.csv"

if not sample_path.exists():
    print(f"Could not find sample data at {sample_path}")
else:
    full_df = pd.read_csv(sample_path)
    
    raw_df = full_df[full_df['city'] == 1].sort_values('time').tail(7).reset_index(drop=True)

    print("--- 1. Input Sample (Continuous 7-Day Window for City 1) ---")
    display(raw_df)

    print("\n--- 2. Running Inference ---")
    try:
        preds_df = predict(raw_df)
        print("Inference complete!\n")

        if "actual_temp_max" in preds_df.columns:
            mae = mean_absolute_error(preds_df["actual_temp_max"], preds_df["predicted_temp_max"])
            print(f"Aligned Mean Absolute Error: {mae:.4f}°C")

        cols_to_show = ["time", "city", "predicted_temp_max"]
        if "actual_temp_max" in preds_df.columns:
            cols_to_show.append("actual_temp_max")

        print("\n--- 3. Prediction Results (Chronological) ---")
        display(preds_df[cols_to_show].sort_values("time"))
        
    except Exception as e:
        print(f"Inference failed with error: {e}")
        import traceback
        traceback.print_exc()

--- 1. Input Sample (Continuous 7-Day Window for City 1) ---


,time,weathercode,temperature_2m_max,temperature_2m_min,precipitation_sum,city
0,2026-02-16,71,-2.0,-5.0,0.8,1
1,2026-02-17,73,-0.9,-4.1,1.5,1
2,2026-02-18,71,-0.3,-6.2,0.7,1
3,2026-02-19,3,0.2,-7.8,0.0,1
4,2026-02-20,3,0.6,-7.9,0.0,1


2026-02-20 18:56:05,131 - INFO - Inference passed all validation checks.



--- 2. Running Inference ---
Inference complete!

Aligned Mean Absolute Error: 1.1349°C

--- 3. Prediction Results (Chronological) ---


,time,city,predicted_temp_max,actual_temp_max
0,2026-02-16,1,1.592568,-2.0
1,2026-02-17,1,0.465982,-0.9
2,2026-02-18,1,-0.099012,-0.3
3,2026-02-19,1,0.575736,0.2
4,2026-02-20,1,0.739054,0.6
